# 🧠 Lecture 3: From First Principles to Production Pipeline

## 🎯 Learning Objectives
1. **Experience** raw data loading from URLs (no libraries)
2. **Stress test** DNNs across 3 architectures × 3 datasets (9 models)
3. **Visualize** training dynamics with comprehensive plots
4. **Design** production-ready ML pipeline architecture
5. **Deploy** interactive UI for model comparison

### 🧭 Experiment Matrix: 3×3 Grid
| Architecture \ Dataset | MNIST | Fashion MNIST | CIFAR-10 |
|------------------------|-------|---------------|----------|
| **Simple DNN** (1 hidden) | Model 1 | Model 2 | Model 3 |
| **Medium DNN** (2 hidden) | Model 4 | Model 5 | Model 6 |
| **Deep DNN** (3 hidden) | Model 7 | Model 8 | Model 9 |

---

## 📦 PART 1: Industrial-Grade Data Loading System

In [ ]:
# %% [code]
import os
import gzip
import struct
import tarfile
import pickle as pkl
import urllib.request
from urllib.error import URLError, HTTPError
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split

# For reproducibility
np.random.seed(42)

# Pipeline directory structure
BASE_DIR = Path("./dl_pipeline_lecture3")
DATA_DIR = BASE_DIR / "data"
MODELS_DIR = BASE_DIR / "models"
RESULTS_DIR = BASE_DIR / "results"
VIZ_DIR = BASE_DIR / "visualizations"

for dir_path in [BASE_DIR, DATA_DIR, MODELS_DIR, RESULTS_DIR, VIZ_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

print("🏗️  Pipeline structure created")
print(f"   Data: {DATA_DIR}")
print(f"   Models: {MODELS_DIR}")
print(f"   Results: {RESULTS_DIR}")
print(f"   Visualizations: {VIZ_DIR}")

In [ ]:
# %% [code]
# -------------------------------------------------------------------
# 📌 DATASETS with multiple mirrors for reliability
# -------------------------------------------------------------------
DATASETS = {
    "mnist": {
        "train_images": [
            "https://storage.googleapis.com/cvdf-datasets/mnist/train-images-idx3-ubyte.gz",
            "http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz",
        ],
        "train_labels": [
            "https://storage.googleapis.com/cvdf-datasets/mnist/train-labels-idx1-ubyte.gz",
            "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz",
        ],
        "test_images": [
            "https://storage.googleapis.com/cvdf-datasets/mnist/t10k-images-idx3-ubyte.gz",
            "http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz",
        ],
        "test_labels": [
            "https://storage.googleapis.com/cvdf-datasets/mnist/t10k-labels-idx1-ubyte.gz",
            "http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz",
        ],
    },
    "fashion": {
        "train_images": ["http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz"],
        "train_labels": ["http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz"],
        "test_images": ["http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz"],
        "test_labels": ["http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz"],
    },
    "cifar10": {
        "train_batch": ["https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"],
    }
}

# -------------------------------------------------------------------
# 📌 Download with retries and progress bar
# -------------------------------------------------------------------
def download_with_retry(url_list, out_path, retries=3):
    """Robust download with progress bar and retry logic"""
    for url in url_list:
        for attempt in range(retries):
            try:
                print(f"   Downloading: {os.path.basename(out_path)} (attempt {attempt+1}/{retries})")
                
                with urllib.request.urlopen(url) as response:
                    total = int(response.headers.get("Content-Length", 0))
                    with open(out_path, "wb") as f, tqdm(
                        total=total, unit="B", unit_scale=True, 
                        desc=os.path.basename(out_path)[:30]
                    ) as bar:
                        while True:
                            chunk = response.read(8192)
                            if not chunk:
                                break
                            f.write(chunk)
                            bar.update(len(chunk))
                
                print("     ✓ Download successful")
                return True
                
            except Exception as e:
                print(f"     ⚠ Error: {e}\n     Retrying...")
    
    print(f"     ❌ Failed to download: {url_list}")
    return False

# -------------------------------------------------------------------
# 📌 Parse CIFAR-10 files
# -------------------------------------------------------------------
def load_cifar10_batch(file_path):
    """Load single CIFAR-10 batch"""
    with open(file_path, 'rb') as f:
        batch = pkl.load(f, encoding='latin1')
        X = batch['data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
        y = np.array(batch['labels'])
    return X, y

def load_cifar10(root):
    """Load all CIFAR-10 batches"""
    X_train = []
    y_train = []
    
    # Load training batches
    for i in range(1, 6):
        batch_path = os.path.join(root, f"data_batch_{i}")
        X_batch, y_batch = load_cifar10_batch(batch_path)
        X_train.append(X_batch)
        y_train.append(y_batch)
    
    # Concatenate
    X_train = np.concatenate(X_train)
    y_train = np.concatenate(y_train)
    
    # Load test batch
    test_path = os.path.join(root, "test_batch")
    X_test, y_test = load_cifar10_batch(test_path)
    
    # CIFAR-10 classes
    cifar10_classes = [
        'airplane', 'automobile', 'bird', 'cat', 'deer',
        'dog', 'frog', 'horse', 'ship', 'truck'
    ]
    
    return (X_train, y_train), (X_test, y_test), cifar10_classes

# -------------------------------------------------------------------
# 📌 Industrial Data Loader
# -------------------------------------------------------------------
class IndustrialDataLoader:
    """Load datasets directly from URLs with caching"""
    
    def __init__(self, dataset="mnist"):
        assert dataset in DATASETS, f"Unknown dataset. Available: {list(DATASETS.keys())}"
        self.dataset = dataset
        self.root = DATA_DIR / dataset
        self.root.mkdir(parents=True, exist_ok=True)
        
        # Class names for each dataset
        if dataset == "mnist":
            self.classes = [str(i) for i in range(10)]
        elif dataset == "fashion":
            self.classes = [
                "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
                "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
            ]
        elif dataset == "cifar10":
            self.classes = [
                'airplane', 'automobile', 'bird', 'cat', 'deer',
                'dog', 'frog', 'horse', 'ship', 'truck'
            ]

    def load(self):
        """Load dataset with progress indication"""
        print(f"📥 Loading {self.dataset.upper()} from source URLs...")
        
        if self.dataset in ["mnist", "fashion"]:
            return self._load_mnist_style()
        else:  # cifar10
            return self._load_cifar10()
    
    def _load_mnist_style(self):
        """Load MNIST or Fashion MNIST"""
        # Download files
        for file_type, urls in DATASETS[self.dataset].items():
            file_path = self.root / f"{file_type}.gz"
            if not file_path.exists():
                download_with_retry(urls, file_path)
        
        # Parse IDX format
        def load_idx_images(filename):
            with gzip.open(filename, 'rb') as f:
                magic, num, rows, cols = struct.unpack('>IIII', f.read(16))
                images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num, rows, cols)
            return images
        
        def load_idx_labels(filename):
            with gzip.open(filename, 'rb') as f:
                magic, num = struct.unpack('>II', f.read(8))
                labels = np.frombuffer(f.read(), dtype=np.uint8)
            return labels
        
        # Load data
        X_train = load_idx_images(self.root / "train_images.gz")
        y_train = load_idx_labels(self.root / "train_labels.gz")
        X_test = load_idx_images(self.root / "test_images.gz")
        y_test = load_idx_labels(self.root / "test_labels.gz")
        
        print(f"     ✓ {self.dataset.upper()}: {X_train.shape[0]:,} train, {X_test.shape[0]:,} test")
        return (X_train, y_train), (X_test, y_test), self.classes
    
    def _load_cifar10(self):
        """Load CIFAR-10"""
        # Download and extract
        tar_path = self.root / "cifar-10-python.tar.gz"
        extracted_path = self.root / "cifar-10-batches-py"
        
        if not tar_path.exists():
            download_with_retry(DATASETS["cifar10"]["train_batch"], tar_path)
        
        if not extracted_path.exists():
            print("     Extracting CIFAR-10...")
            with tarfile.open(tar_path, 'r:gz') as tar:
                tar.extractall(path=self.root)
        
        # Load data
        (X_train, y_train), (X_test, y_test), _ = load_cifar10(extracted_path)
        
        print(f"     ✓ CIFAR-10: {X_train.shape[0]:,} train, {X_test.shape[0]:,} test")
        return (X_train, y_train), (X_test, y_test), self.classes

# -------------------------------------------------------------------
# 📌 Load and visualize all datasets
# -------------------------------------------------------------------
print("\n📊 LOADING ALL 3 DATASETS FROM SOURCE URLs")
print("=" * 60)

all_datasets = {}
for dataset_name in ["mnist", "fashion", "cifar10"]:
    loader = IndustrialDataLoader(dataset=dataset_name)
    train_data, test_data, classes = loader.load()
    all_datasets[dataset_name] = {
        'train': train_data,
        'test': test_data,
        'classes': classes
    }

# Visualize dataset complexity
fig, axes = plt.subplots(3, 5, figsize=(15, 9))

for row_idx, (dataset_name, data_info) in enumerate(all_datasets.items()):
    X_train, y_train = data_info['train']
    classes = data_info['classes']
    
    # Show 5 random samples per dataset
    indices = np.random.choice(len(X_train), 5, replace=False)
    
    for col_idx, idx in enumerate(indices):
        ax = axes[row_idx, col_idx]
        img = X_train[idx]
        
        if dataset_name == "cifar10":
            ax.imshow(img.astype(np.uint8))
        else:
            ax.imshow(img, cmap='gray')
        
        label = y_train[idx]
        ax.set_title(f"{classes[label][:10]}", fontsize=9)
        ax.axis('off')
    
    # Dataset title
    axes[row_idx, 2].text(0.5, 1.2, f"{dataset_name.upper()}\n{X_train.shape[1:]} {X_train.dtype}", 
                          ha='center', va='center', transform=axes[row_idx, 2].transAxes,
                          fontsize=11, fontweight='bold')

plt.suptitle('Dataset Complexity Gradient: MNIST → Fashion MNIST → CIFAR-10', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(VIZ_DIR / "dataset_complexity.png", dpi=150, bbox_inches='tight')
plt.show()

## 🔧 PART 2: Modular Preprocessing Pipeline

In [ ]:
# %% [code]
class DataPreprocessor:
    """Modular preprocessing with configurable transformations"""
    
    def __init__(self, flatten=True, normalize=True, rgb_to_grayscale=False):
        self.flatten = flatten
        self.normalize = normalize
        self.rgb_to_grayscale = rgb_to_grayscale
        
    def transform(self, X, y=None):
        """Apply preprocessing pipeline"""
        X = X.copy().astype(np.float32)
        
        # 1. Convert RGB to grayscale if needed (for DNN comparison)
        if self.rgb_to_grayscale and X.ndim == 4 and X.shape[-1] == 3:
            X = np.mean(X, axis=-1, keepdims=True)
            
        # 2. Normalize to [0, 1]
        if self.normalize:
            X = X / 255.0
            
        # 3. Flatten for DNN
        if self.flatten and X.ndim > 2:
            original_shape = X.shape
            X = X.reshape(X.shape[0], -1)
            
        # 4. One-hot encode labels if provided
        if y is not None:
            y = y.astype(int)
            n_classes = len(np.unique(y))
            y_onehot = np.zeros((len(y), n_classes))
            y_onehot[np.arange(len(y)), y] = 1
            return X, y_onehot
        
        return X
    
    def prepare_dataset(self, X_train, y_train, X_test, y_test, val_size=0.1):
        """Complete dataset preparation"""
        # Transform
        X_train_proc, y_train_proc = self.transform(X_train, y_train)
        X_test_proc, y_test_proc = self.transform(X_test, y_test)
        
        # Create validation split
        X_train_final, X_val, y_train_final, y_val = train_test_split(
            X_train_proc, y_train_proc, 
            test_size=val_size, 
            random_state=42,
            stratify=y_train
        )
        
        print(f"   Prepared: {X_train.shape} → {X_train_final.shape} (train)")
        print(f"             {X_test.shape} → {X_test_proc.shape} (test)")
        print(f"             {X_val.shape} (validation)")
        
        return {
            'X_train': X_train_final, 'y_train': y_train_final,
            'X_val': X_val, 'y_val': y_val,
            'X_test': X_test_proc, 'y_test': y_test_proc
        }

# Prepare all datasets
print("\n🔧 PREPROCESSING ALL DATASETS")
print("=" * 60)

prepared_data = {}
for dataset_name, data_info in all_datasets.items():
    print(f"\nProcessing {dataset_name.upper()}:")
    
    X_train, y_train = data_info['train']
    X_test, y_test = data_info['test']
    
    # Special preprocessing for CIFAR-10
    if dataset_name == "cifar10":
        preprocessor = DataPreprocessor(flatten=True, rgb_to_grayscale=True)
    else:
        preprocessor = DataPreprocessor(flatten=True)
    
    data_dict = preprocessor.prepare_dataset(X_train, y_train, X_test, y_test)
    prepared_data[dataset_name] = {
        'data': data_dict,
        'classes': data_info['classes'],
        'preprocessor': preprocessor,
        'original_shape': X_train.shape[1:]
    }

print("\n✅ All datasets preprocessed and ready for training")

## 🏗️ PART 3: Neural Network Components (First Principles)

In [ ]:
# %% [code]
import pickle
import time
from tqdm import tqdm

class NeuralMath:
    """Pure mathematical operations - no state"""
    
    @staticmethod
    def relu(x):
        return np.maximum(0, x)
    
    @staticmethod
    def relu_derivative(x):
        return (x > 0).astype(np.float32)
    
    @staticmethod
    def softmax(x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)
    
    @staticmethod
    def cross_entropy(y_pred, y_true):
        y_pred = np.clip(y_pred, 1e-12, 1 - 1e-12)
        return -np.mean(np.sum(y_true * np.log(y_pred), axis=1))
    
    @staticmethod
    def cross_entropy_gradient(y_pred, y_true):
        return y_pred - y_true
    
    @staticmethod
    def initialize_weights(shape, activation='relu'):
        """Smart initialization"""
        if activation == 'relu':
            # He initialization
            std = np.sqrt(2.0 / shape[0])
        else:
            # Xavier initialization
            std = np.sqrt(2.0 / (shape[0] + shape[1]))
        return np.random.randn(*shape) * std

class DenseLayer:
    """Single neural network layer"""
    
    def __init__(self, input_size, output_size, activation='relu', l2_lambda=0.0001):
        self.input_size = input_size
        self.output_size = output_size
        self.activation = activation
        self.l2_lambda = l2_lambda
        
        # Initialize parameters
        self.weights = NeuralMath.initialize_weights((input_size, output_size), activation)
        self.biases = np.zeros((1, output_size))
        
        # Cache for backprop
        self.input_cache = None
        self.output_cache = None
        
    def forward(self, X):
        """Forward pass"""
        self.input_cache = X
        Z = X @ self.weights + self.biases
        
        if self.activation == 'relu':
            A = NeuralMath.relu(Z)
        elif self.activation == 'softmax':
            A = NeuralMath.softmax(Z)
        else:
            A = Z
        
        self.output_cache = A
        return A
    
    def backward(self, dL_dA, learning_rate):
        """Backward pass"""
        batch_size = self.input_cache.shape[0]
        
        if self.activation == 'relu':
            dA_dZ = NeuralMath.relu_derivative(self.output_cache)
            dL_dZ = dL_dA * dA_dZ
        elif self.activation == 'softmax':
            dL_dZ = dL_dA
        else:
            dL_dZ = dL_dA
        
        # Compute gradients
        dL_dW = (self.input_cache.T @ dL_dZ) / batch_size
        dL_db = np.sum(dL_dZ, axis=0, keepdims=True) / batch_size
        
        # Add L2 regularization
        if self.l2_lambda > 0:
            dL_dW += self.l2_lambda * self.weights / batch_size
        
        # Update parameters
        self.weights -= learning_rate * dL_dW
        self.biases -= learning_rate * dL_db
        
        # Gradient for previous layer
        dL_dinput = dL_dZ @ self.weights.T
        return dL_dinput
    
    @property
    def num_params(self):
        return self.weights.size + self.biases.size

class DNN:
    """Deep Neural Network with training visualization"""
    
    def __init__(self, layer_sizes, activations, learning_rate=0.001, 
                 l2_lambda=0.0001, name="DNN"):
        self.layer_sizes = layer_sizes
        self.activations = activations
        self.learning_rate = learning_rate
        self.l2_lambda = l2_lambda
        self.name = name
        
        # Build layers
        self.layers = []
        for i in range(len(layer_sizes) - 1):
            layer = DenseLayer(
                input_size=layer_sizes[i],
                output_size=layer_sizes[i + 1],
                activation=activations[i],
                l2_lambda=l2_lambda
            )
            self.layers.append(layer)
        
        # Training history
        self.history = {
            'train_loss': [], 'train_acc': [],
            'val_loss': [], 'val_acc': [],
            'epoch_times': [],
            'learning_rates': []
        }
        
        print(f"🧠 Built {name}: {layer_sizes}")
        print(f"   Parameters: {self.num_params:,}")
    
    @property
    def num_params(self):
        return sum(layer.num_params for layer in self.layers)
    
    def forward(self, X, training=True):
        """Forward pass through all layers"""
        activations = X
        for layer in self.layers:
            activations = layer.forward(activations)
        return activations
    
    def compute_loss(self, y_pred, y_true):
        """Compute total loss"""
        loss = NeuralMath.cross_entropy(y_pred, y_true)
        
        # Add L2 regularization
        if self.l2_lambda > 0:
            reg_loss = 0
            for layer in self.layers:
                reg_loss += np.sum(layer.weights ** 2)
            loss += (self.l2_lambda / (2 * y_true.shape[0])) * reg_loss
        
        return loss
    
    def compute_accuracy(self, y_pred, y_true):
        """Compute accuracy"""
        pred_labels = np.argmax(y_pred, axis=1)
        true_labels = np.argmax(y_true, axis=1)
        return np.mean(pred_labels == true_labels)
    
    def train_epoch(self, X_batch, y_batch):
        """Single training step"""
        # Forward pass
        y_pred = self.forward(X_batch)
        
        # Compute gradient
        loss_grad = NeuralMath.cross_entropy_gradient(y_pred, y_batch)
        
        # Backward pass
        grad = loss_grad
        for layer in reversed(self.layers):
            grad = layer.backward(grad, self.learning_rate)
        
        # Compute metrics
        loss = self.compute_loss(y_pred, y_batch)
        accuracy = self.compute_accuracy(y_pred, y_batch)
        
        return loss, accuracy
    
    def train(self, X_train, y_train, X_val=None, y_val=None, 
              epochs=20, batch_size=64, verbose=True):
        """Training loop with progress bars"""
        n_samples = X_train.shape[0]
        n_batches = int(np.ceil(n_samples / batch_size))
        
        print(f"🚀 Training {self.name} for {epochs} epochs")
        print(f"   Samples: {n_samples:,}, Batch size: {batch_size}, Batches/epoch: {n_batches}")
        print("-" * 70)
        
        for epoch in range(epochs):
            epoch_start = time.time()
            epoch_loss, epoch_acc = 0, 0
            
            # Shuffle data
            indices = np.random.permutation(n_samples)
            X_shuffled = X_train[indices]
            y_shuffled = y_train[indices]
            
            # Mini-batch training with progress bar
            with tqdm(total=n_batches, desc=f"Epoch {epoch+1}/{epochs}", 
                     bar_format='{l_bar}{bar:30}{r_bar}{bar:-30b}',
                     leave=False) as pbar:
                for batch in range(n_batches):
                    start = batch * batch_size
                    end = min(start + batch_size, n_samples)
                    
                    X_batch = X_shuffled[start:end]
                    y_batch = y_shuffled[start:end]
                    
                    batch_loss, batch_acc = self.train_epoch(X_batch, y_batch)
                    epoch_loss += batch_loss
                    epoch_acc += batch_acc
                    
                    # Update progress bar
                    pbar.set_postfix({
                        'loss': f'{batch_loss:.4f}',
                        'acc': f'{batch_acc:.2%}'
                    })
                    pbar.update(1)
            
            # Average over batches
            epoch_loss /= n_batches
            epoch_acc /= n_batches
            
            # Validation
            if X_val is not None and y_val is not None:
                y_val_pred = self.forward(X_val)
                val_loss = self.compute_loss(y_val_pred, y_val)
                val_acc = self.compute_accuracy(y_val_pred, y_val)
            else:
                val_loss, val_acc = None, None
            
            # Record history
            self.history['train_loss'].append(epoch_loss)
            self.history['train_acc'].append(epoch_acc)
            self.history['epoch_times'].append(time.time() - epoch_start)
            self.history['learning_rates'].append(self.learning_rate)
            
            if val_loss is not None:
                self.history['val_loss'].append(val_loss)
                self.history['val_acc'].append(val_acc)
            
            # Progress report
            if verbose:
                if val_loss is not None:
                    print(f"Epoch {epoch+1:3d} | Train: loss={epoch_loss:.4f}, acc={epoch_acc:.2%} | "
                          f"Val: loss={val_loss:.4f}, acc={val_acc:.2%} | Time: {self.history['epoch_times'][-1]:.1f}s")
                else:
                    print(f"Epoch {epoch+1:3d} | Train: loss={epoch_loss:.4f}, acc={epoch_acc:.2%} | "
                          f"Time: {self.history['epoch_times'][-1]:.1f}s")
        
        print("-" * 70)
        avg_time = np.mean(self.history['epoch_times'])
        print(f"✅ Training complete. Avg time/epoch: {avg_time:.2f}s")
    
    def evaluate(self, X, y):
        """Evaluate model"""
        y_pred = self.forward(X)
        loss = self.compute_loss(y_pred, y)
        accuracy = self.compute_accuracy(y_pred, y)
        return loss, accuracy
    
    def predict(self, X):
        """Make predictions"""
        y_pred = self.forward(X)
        return np.argmax(y_pred, axis=1), y_pred
    
    def save(self, filename):
        """Save model to disk"""
        with open(filename, 'wb') as f:
            pickle.dump(self, f)
        print(f"💾 Saved: {filename}")
    
    @staticmethod
    def load(filename):
        """Load model from disk"""
        with open(filename, 'rb') as f:
            model = pickle.load(f)
        print(f"📂 Loaded: {filename}")
        return model

## 📊 PART 4: Training Visualization System

In [ ]:
# %% [code]
class TrainingVisualizer:
    """Comprehensive training visualization"""
    
    @staticmethod
    def plot_training_history(history, model_name, dataset_name, save_path=None):
        """Create 2x2 visualization grid"""
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        epochs = range(1, len(history['train_loss']) + 1)
        
        # 1. Loss curves
        ax = axes[0, 0]
        ax.plot(epochs, history['train_loss'], 'b-', label='Train Loss', linewidth=2)
        if history['val_loss']:
            ax.plot(epochs, history['val_loss'], 'r-', label='Val Loss', linewidth=2)
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Loss')
        ax.set_title('Loss Curves')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # 2. Accuracy curves
        ax = axes[0, 1]
        ax.plot(epochs, history['train_acc'], 'b-', label='Train Acc', linewidth=2)
        if history['val_acc']:
            ax.plot(epochs, history['val_acc'], 'r-', label='Val Acc', linewidth=2)
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Accuracy')
        ax.set_title('Accuracy Curves')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # 3. Training time per epoch
        ax = axes[1, 0]
        ax.plot(epochs, history['epoch_times'], 'purple', marker='o', alpha=0.7)
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Time (seconds)')
        ax.set_title('Training Time per Epoch')
        ax.grid(True, alpha=0.3)
        
        # 4. Loss-Accuracy tradeoff
        ax = axes[1, 1]
        scatter = ax.scatter(history['train_loss'], history['train_acc'], 
                           c=epochs, cmap='viridis', s=50, alpha=0.7)
        ax.set_xlabel('Loss')
        ax.set_ylabel('Accuracy')
        ax.set_title('Loss vs Accuracy (Epochs)')
        ax.grid(True, alpha=0.3)
        plt.colorbar(scatter, ax=ax, label='Epoch')
        
        plt.suptitle(f'{model_name} on {dataset_name}', 
                    fontsize=16, fontweight='bold', y=1.02)
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.show()
    
    @staticmethod
    def plot_all_models_comparison(all_results):
        """Compare all 9 models"""
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        # Extract data
        model_names = []
        datasets = []
        test_accuracies = []
        parameters = []
        
        for (model_name, dataset_name), results in all_results.items():
            model_names.append(model_name)
            datasets.append(dataset_name)
            test_accuracies.append(results['test_accuracy'])
            parameters.append(results['parameters'])
        
        # Color by dataset
        dataset_colors = {'mnist': 'blue', 'fashion': 'orange', 'cifar10': 'green'}
        colors = [dataset_colors[d] for d in datasets]
        
        # 1. Test accuracy comparison
        x_pos = np.arange(len(model_names))
        bars = axes[0].bar(x_pos, test_accuracies, color=colors, edgecolor='black')
        axes[0].set_xlabel('Model')
        axes[0].set_ylabel('Test Accuracy')
        axes[0].set_title('Test Accuracy Comparison (9 Models)')
        axes[0].set_xticks(x_pos)
        axes[0].set_xticklabels([f"{m}\n({d})" for m, d in zip(model_names, datasets)], 
                               rotation=45, ha='right', fontsize=8)
        axes[0].set_ylim([0, 1])
        axes[0].grid(True, alpha=0.3, axis='y')
        
        # Add accuracy labels
        for bar, acc in zip(bars, test_accuracies):
            height = bar.get_height()
            axes[0].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                       f'{acc:.1%}', ha='center', va='bottom', fontsize=8)
        
        # 2. Parameters vs Accuracy
        scatter = axes[1].scatter(parameters, test_accuracies, s=100, c=colors, 
                                 edgecolor='black', alpha=0.7)
        axes[1].set_xlabel('Number of Parameters')
        axes[1].set_ylabel('Test Accuracy')
        axes[1].set_title('Parameters vs Accuracy Efficiency')
        axes[1].grid(True, alpha=0.3)
        axes[1].set_xscale('log')
        
        # Add labels for each point
        for i, (model, dataset, param, acc) in enumerate(zip(model_names, datasets, parameters, test_accuracies)):
            axes[1].annotate(f"{model}\n{dataset}", 
                           (param, acc), 
                           xytext=(5, 5), 
                           textcoords='offset points',
                           fontsize=7,
                           bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.7))
        
        # 3. Dataset performance comparison
        ax = axes[2]
        dataset_means = {}
        for dataset in ['mnist', 'fashion', 'cifar10']:
            dataset_accs = [acc for d, acc in zip(datasets, test_accuracies) if d == dataset]
            if dataset_accs:
                dataset_means[dataset] = np.mean(dataset_accs)
        
        bars_ds = ax.bar(range(len(dataset_means)), list(dataset_means.values()), 
                        color=[dataset_colors[d] for d in dataset_means.keys()], 
                        edgecolor='black')
        ax.set_xlabel('Dataset')
        ax.set_ylabel('Average Test Accuracy')
        ax.set_title('Dataset Difficulty Comparison')
        ax.set_xticks(range(len(dataset_means)))
        ax.set_xticklabels([f"{d.upper()}\n({len([x for x in datasets if x == d])} models)" 
                           for d in dataset_means.keys()])
        ax.set_ylim([0, 1])
        ax.grid(True, alpha=0.3, axis='y')
        
        for bar, acc in zip(bars_ds, dataset_means.values()):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                   f'{acc:.1%}', ha='center', va='bottom', fontsize=9)
        
        plt.suptitle('3×3 Experiment Matrix: 3 Architectures × 3 Datasets', 
                    fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig(VIZ_DIR / 'all_models_comparison.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        # Print summary table
        print("\n📊 SUMMARY TABLE: 3×3 EXPERIMENT MATRIX")
        print("=" * 85)
        print(f"{'Model':<20} {'Dataset':<15} {'Test Acc':<12} {'Params':<15} {'Params/Acc':<15}")
        print("-" * 85)
        
        for (model_name, dataset_name), results in all_results.items():
            params = results['parameters']
            acc = results['test_accuracy']
            efficiency = params / (acc + 1e-8)
            
            print(f"{model_name:<20} {dataset_name:<15} {acc:<12.2%} {params:<15,} {efficiency:,.0f}")
        
        print("-" * 85)

## 🧪 PART 5: 3×3 Experiment Matrix (9 Models)

In [ ]:
# %% [code]
# Define 3 architectures
ARCHITECTURES = {
    'simple_dnn': {
        'layer_sizes': [None, 64, 10],  # None will be replaced with input size
        'activations': ['relu', 'softmax'],
        'learning_rate': 0.001,
        'l2_lambda': 0.0001,
        'epochs': 20
    },
    'medium_dnn': {
        'layer_sizes': [None, 128, 64, 10],
        'activations': ['relu', 'relu', 'softmax'],
        'learning_rate': 0.0005,
        'l2_lambda': 0.0005,
        'epochs': 25
    },
    'deep_dnn': {
        'layer_sizes': [None, 256, 128, 64, 10],
        'activations': ['relu', 'relu', 'relu', 'softmax'],
        'learning_rate': 0.0003,
        'l2_lambda': 0.001,
        'epochs': 30
    }
}

print("\n🔬 RUNNING 3×3 EXPERIMENT MATRIX (9 MODELS)")
print("=" * 80)

all_models = {}
all_results = {}

for arch_name, arch_config in ARCHITECTURES.items():
    print(f"\n🏗️  ARCHITECTURE: {arch_name.upper()}")
    print("-" * 60)
    
    for dataset_name in ['mnist', 'fashion', 'cifar10']:
        # Get data
        data_dict = prepared_data[dataset_name]['data']
        input_size = data_dict['X_train'].shape[1]
        
        # Update config with input size
        config = arch_config.copy()
        config['layer_sizes'][0] = input_size
        
        # Create model
        model_name = f"{arch_name}_{dataset_name}"
        model = DNN(
            layer_sizes=config['layer_sizes'],
            activations=config['activations'],
            learning_rate=config['learning_rate'],
            l2_lambda=config['l2_lambda'],
            name=model_name
        )
        
        print(f"\n📊 Training {model_name}:")
        
        # Train model
        model.train(
            X_train=data_dict['X_train'],
            y_train=data_dict['y_train'],
            X_val=data_dict['X_val'],
            y_val=data_dict['y_val'],
            epochs=config['epochs'],
            batch_size=64,
            verbose=True
        )
        
        # Evaluate
        test_loss, test_acc = model.evaluate(data_dict['X_test'], data_dict['y_test'])
        
        # Store results
        all_models[model_name] = model
        all_results[(arch_name, dataset_name)] = {
            'model': model_name,
            'test_accuracy': test_acc,
            'test_loss': test_loss,
            'parameters': model.num_params,
            'history': model.history
        }
        
        print(f"   Test Accuracy: {test_acc:.2%}, Test Loss: {test_loss:.4f}")
        
        # Save model
        model.save(MODELS_DIR / f"{model_name}.pkl")
        
        # Visualize training
        TrainingVisualizer.plot_training_history(
            model.history,
            model_name=model_name,
            dataset_name=dataset_name.upper(),
            save_path=VIZ_DIR / f"training_{model_name}.png"
        )

print("\n✅ ALL 9 MODELS TRAINED AND SAVED!")

## 📈 PART 6: Comprehensive Analysis

In [ ]:
# %% [code]
# Compare all models
TrainingVisualizer.plot_all_models_comparison(all_results)

# Key insights
print("\n🔍 KEY INSIGHTS FROM 3×3 EXPERIMENT")
print("=" * 80)

# Calculate dataset averages
dataset_stats = {}
for dataset in ['mnist', 'fashion', 'cifar10']:
    dataset_accs = [results['test_accuracy'] for (arch, ds), results in all_results.items() if ds == dataset]
    dataset_stats[dataset] = {
        'mean_accuracy': np.mean(dataset_accs),
        'std_accuracy': np.std(dataset_accs),
        'best_accuracy': np.max(dataset_accs),
        'worst_accuracy': np.min(dataset_accs)
    }

# Print insights
insights = [
    ("📉 Performance Drop with Complexity",
     f"MNIST: {dataset_stats['mnist']['mean_accuracy']:.1%} avg → "
     f"Fashion MNIST: {dataset_stats['fashion']['mean_accuracy']:.1%} avg → "
     f"CIFAR-10: {dataset_stats['cifar10']['mean_accuracy']:.1%} avg\n"
     "   DNNs struggle as data complexity increases."),
    
    ("🏗️ Diminishing Returns with Depth",
     "Deep DNN (3 hidden) is only ~5-10% better than Simple DNN (1 hidden)\n"
     "   but has 3-4x more parameters → poor parameter efficiency."),
    
    ("🎨 RGB → Grayscale Information Loss",
     "CIFAR-10 converted to grayscale loses critical color information.\n"
     "   A red car vs blue car looks identical to DNN."),
    
    ("🔄 Translation Sensitivity",
     "A shirt in top-left vs bottom-right appears completely different.\n"
     "   DNNs have no built-in translation invariance."),
    
    ("📊 Parameter Explosion",
     f"CIFAR-10 input: 32×32×3 = 3,072 pixels → ~{all_models['deep_dnn_cifar10'].num_params:,} params\n"
     "   MNIST input: 28×28 = 784 pixels → much fewer params\n"
     "   Yet accuracy is lower with more parameters!"),
    
    ("💡 Why CNNs Are Needed",
     "• Parameter sharing → efficiency\n"
     "• Translation invariance → robustness\n"
     "• Hierarchical features → better learning\n"
     "• Spatial preservation → understand relationships")
]

for i, (title, content) in enumerate(insights, 1):
    print(f"\n{i}. {title}")
    print(content.replace('\n', '\n   '))

print("\n" + "=" * 80)
print("🎯 CONCLUSION: DNNs work for simple patterns but fail dramatically\n"
      "          for complex spatial data. This clearly demonstrates why\n"
      "          Convolutional Neural Networks (CNNs) were invented.")
print("=" * 80)

## 🏗️ PART 7: Pipeline Visualization System

In [ ]:
# %% [code]
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.patches import FancyBboxPatch, Circle

class PipelineVisualizer:
    """Visualize deep learning pipeline architecture"""
    
    @staticmethod
    def visualize_complete_pipeline():
        """Create comprehensive pipeline visualization"""
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        
        # 1. Data Flow Diagram
        ax1 = axes[0, 0]
        PipelineVisualizer._visualize_data_flow(ax1)
        
        # 2. Training Pipeline
        ax2 = axes[0, 1]
        PipelineVisualizer._visualize_training_pipeline(ax2)
        
        # 3. Model Architecture
        ax3 = axes[0, 2]
        PipelineVisualizer._visualize_model_architecture(ax3)
        
        # 4. System Components
        ax4 = axes[1, 0]
        PipelineVisualizer._visualize_system_components(ax4)
        
        # 5. Production Deployment
        ax5 = axes[1, 1]
        PipelineVisualizer._visualize_deployment_pipeline(ax5)
        
        # 6. Monitoring & Maintenance
        ax6 = axes[1, 2]
        PipelineVisualizer._visualize_monitoring_pipeline(ax6)
        
        plt.suptitle("Deep Learning Pipeline: Complete System Architecture", 
                    fontsize=16, fontweight='bold', y=1.02)
        plt.tight_layout()
        plt.savefig(VIZ_DIR / "pipeline_architecture.png", dpi=150, bbox_inches='tight')
        plt.show()
    
    @staticmethod
    def _visualize_data_flow(ax):
        """Visualize data flow through pipeline"""
        stages = [
            "Raw Data",
            "Data Loader",
            "Preprocessor",
            "Augmentations",
            "Data Splits",
            "Data Loaders"
        ]
        
        colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7', '#DDA0DD']
        
        for i, (stage, color) in enumerate(zip(stages, colors)):
            # Create rounded rectangle
            bbox = FancyBboxPatch((0.1, 0.8 - i*0.15), 0.8, 0.1,
                                 boxstyle="round,pad=0.02",
                                 facecolor=color, edgecolor='black',
                                 linewidth=2)
            ax.add_patch(bbox)
            ax.text(0.5, 0.85 - i*0.15, stage,
                   ha='center', va='center', fontsize=10, fontweight='bold')
            
            # Add arrows
            if i < len(stages) - 1:
                ax.arrow(0.5, 0.75 - i*0.15, 0, -0.1,
                        head_width=0.03, head_length=0.02,
                        fc='black', ec='black')
        
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.set_title("📊 Data Flow Pipeline", fontsize=12, fontweight='bold')
        ax.axis('off')
    
    @staticmethod
    def _visualize_training_pipeline(ax):
        """Visualize training pipeline"""
        # Create directed graph
        G = nx.DiGraph()
        
        nodes = [
            "Data Loader", "Model", "Loss",
            "Optimizer", "Backward Pass", "Parameter Update",
            "Validation", "Checkpoint", "Logging"
        ]
        
        edges = [
            ("Data Loader", "Model"),
            ("Model", "Loss"),
            ("Loss", "Backward Pass"),
            ("Backward Pass", "Optimizer"),
            ("Optimizer", "Parameter Update"),
            ("Parameter Update", "Model"),  # Next iteration
            ("Model", "Validation"),
            ("Validation", "Checkpoint"),
            ("Validation", "Logging")
        ]
        
        G.add_nodes_from(nodes)
        G.add_edges_from(edges)
        
        # Position nodes
        pos = {
            "Data Loader": (0.2, 0.8),
            "Model": (0.5, 0.8),
            "Loss": (0.8, 0.8),
            "Backward Pass": (0.8, 0.6),
            "Optimizer": (0.5, 0.6),
            "Parameter Update": (0.2, 0.6),
            "Validation": (0.5, 0.4),
            "Checkpoint": (0.2, 0.4),
            "Logging": (0.8, 0.4)
        }
        
        # Draw graph
        nx.draw_networkx_nodes(G, pos, node_color='lightblue', 
                              node_size=2000, ax=ax, alpha=0.8)
        nx.draw_networkx_edges(G, pos, edge_color='gray', 
                              arrows=True, arrowsize=20, ax=ax, width=2)
        nx.draw_networkx_labels(G, pos, font_size=8, ax=ax, font_weight='bold')
        
        ax.set_title("🚀 Training Pipeline", fontsize=12, fontweight='bold')
        ax.axis('off')
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
    
    @staticmethod
    def _visualize_model_architecture(ax):
        """Visualize model architecture"""
        # Create a simple neural network visualization
        layers = [4, 8, 6, 2]  # Layer sizes
        
        # Calculate positions
        x_positions = np.linspace(0.1, 0.9, len(layers))
        
        for layer_idx, (x, n_neurons) in enumerate(zip(x_positions, layers)):
            y_positions = np.linspace(0.1, 0.9, n_neurons)
            
            for y in y_positions:
                circle = Circle((x, y), 0.03, color='skyblue', ec='black', lw=1)
                ax.add_patch(circle)
            
            # Draw layer label
            layer_type = ["Input", "Hidden", "Hidden", "Output"][layer_idx]
            ax.text(x, 0.05, f"{layer_type}\n({n_neurons})", 
                   ha='center', va='center', fontsize=9)
        
        # Draw connections between layers
        for i in range(len(layers) - 1):
            x1 = x_positions[i]
            x2 = x_positions[i + 1]
            y1_positions = np.linspace(0.1, 0.9, layers[i])
            y2_positions = np.linspace(0.1, 0.9, layers[i + 1])
            
            # Draw a few sample connections
            for y1 in y1_positions[:3]:  # Only show first 3 neurons' connections
                for y2 in y2_positions[:3]:
                    ax.plot([x1, x2], [y1, y2], 'gray', alpha=0.3, linewidth=0.5)
        
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.set_title("🧠 Model Architecture", fontsize=12, fontweight='bold')
        ax.axis('off')
    
    @staticmethod
    def _visualize_system_components(ax):
        """Visualize system component relationships"""
        components = {
            "Core": ["Data Module", "Model Module", "Training Module"],
            "Support": ["Config Manager", "Logger", "Monitor"],
            "Infrastructure": ["Experiment Tracker", "Model Registry", "Serving Engine"]
        }
        
        y_pos = 0.9
        for category, items in components.items():
            ax.text(0.1, y_pos, f"{category}:", 
                   fontsize=10, fontweight='bold', va='center')
            
            for i, item in enumerate(items):
                ax.text(0.3, y_pos - (i+1)*0.08, f"• {item}", 
                       fontsize=9, va='center')
            
            y_pos -= (len(items) + 1) * 0.08
        
        # Add connections
        ax.plot([0.6, 0.8], [0.7, 0.7], 'k-', alpha=0.5)
        ax.plot([0.6, 0.8], [0.5, 0.5], 'k-', alpha=0.5)
        ax.plot([0.6, 0.8], [0.3, 0.3], 'k-', alpha=0.5)
        
        ax.text(0.85, 0.7, "API Layer", fontsize=9, ha='center', 
               bbox=dict(boxstyle="round,pad=0.3", facecolor="lightyellow"))
        
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.set_title("🏗️ System Components", fontsize=12, fontweight='bold')
        ax.axis('off')
    
    @staticmethod
    def _visualize_deployment_pipeline(ax):
        """Visualize production deployment pipeline"""
        stages = [
            ("Model Training", 0.2, 0.9),
            ("Model Validation", 0.5, 0.9),
            ("Model Export", 0.8, 0.9),
            ("Containerization", 0.2, 0.7),
            ("API Development", 0.5, 0.7),
            ("Load Testing", 0.8, 0.7),
            ("Cloud Deployment", 0.2, 0.5),
            ("Auto-scaling", 0.5, 0.5),
            ("Monitoring Setup", 0.8, 0.5),
            ("Production", 0.5, 0.3)
        ]
        
        colors = plt.cm.Set3(np.linspace(0, 1, len(stages)))
        
        for (stage, x, y), color in zip(stages, colors):
            # Draw circle for each stage
            circle = Circle((x, y), 0.06, color=color, ec='black', lw=1.5)
            ax.add_patch(circle)
            ax.text(x, y, stage.replace(' ', '\n'), 
                   ha='center', va='center', fontsize=7, fontweight='bold')
        
        # Draw flow arrows
        arrows = [
            ((0.2, 0.83), (0.2, 0.77)),  # Training -> Containerization
            ((0.5, 0.83), (0.5, 0.77)),  # Validation -> API Dev
            ((0.8, 0.83), (0.8, 0.77)),  # Export -> Load Testing
            ((0.2, 0.63), (0.2, 0.57)),  # Containerization -> Cloud
            ((0.5, 0.63), (0.5, 0.57)),  # API Dev -> Auto-scaling
            ((0.8, 0.63), (0.8, 0.57)),  # Load Testing -> Monitoring
            ((0.2, 0.43), (0.4, 0.37)),  # Cloud -> Production
            ((0.5, 0.43), (0.5, 0.37)),  # Auto-scaling -> Production
            ((0.8, 0.43), (0.6, 0.37))   # Monitoring -> Production
        ]
        
        for (x1, y1), (x2, y2) in arrows:
            ax.arrow(x1, y1, x2-x1, y2-y1, 
                    head_width=0.02, head_length=0.03,
                    fc='black', ec='black', alpha=0.7)
        
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.set_title("🚀 Production Deployment", fontsize=12, fontweight='bold')
        ax.axis('off')
    
    @staticmethod
    def _visualize_monitoring_pipeline(ax):
        """Visualize monitoring and maintenance pipeline"""
        # Create a monitoring dashboard visualization
        metrics = ["Accuracy", "Loss", "Latency", "Memory", "CPU", "GPU"]
        values = [0.85, 0.32, 45, 78, 65, 92]  # Example values
        
        # Create gauge-like indicators
        angles = np.linspace(0, 2*np.pi, len(metrics), endpoint=False)
        radius = 0.35
        
        for metric, value, angle in zip(metrics, values, angles):
            # Normalize value for display (0-100%)
            norm_value = value / 100 if metric in ["Memory", "CPU", "GPU"] else value
            
            # Calculate position
            x = 0.5 + radius * np.cos(angle)
            y = 0.5 + radius * np.sin(angle)
            
            # Draw gauge background
            gauge_bg = Circle((x, y), 0.08, color='lightgray', ec='black', alpha=0.3)
            ax.add_patch(gauge_bg)
            
            # Draw gauge fill
            fill_radius = 0.08 * norm_value
            gauge_fill = Circle((x, y), fill_radius, color='green' if norm_value > 0.7 else 
                               'orange' if norm_value > 0.4 else 'red', 
                               alpha=0.7)
            ax.add_patch(gauge_fill)
            
            # Add metric label
            ax.text(x, y - 0.12, metric, ha='center', va='center', fontsize=8)
            ax.text(x, y, f"{value:.1f}{'%' if metric in ['Memory', 'CPU', 'GPU'] else ''}", 
                   ha='center', va='center', fontsize=9, fontweight='bold')
        
        # Add central title
        ax.text(0.5, 0.5, "Live\nMonitoring", ha='center', va='center',
               fontsize=10, fontweight='bold', 
               bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
        
        # Add alerts section
        ax.text(0.15, 0.15, "🔴 High Loss\n🟡 Memory 78%\n🟢 System OK", 
               fontsize=8, bbox=dict(boxstyle="round,pad=0.3", facecolor="white"))
        
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.set_title("📈 Monitoring & Maintenance", fontsize=12, fontweight='bold')
        ax.axis('off')

# Show pipeline visualization
print("\n🏗️ VISUALIZING PRODUCTION PIPELINE ARCHITECTURE")
PipelineVisualizer.visualize_complete_pipeline()

# Pipeline design principles
print("\n📋 PIPELINE DESIGN PRINCIPLES")
print("=" * 60)

principles = [
    "1. Modularity: Each component does one thing well",
    "2. Reproducibility: Same code → same results",
    "3. Testability: Unit test each component",
    "4. Monitoring: Track everything, alert on anomalies",
    "5. Versioning: Data, code, models, and configs",
    "6. Automation: Manual steps are failure points",
    "7. Scalability: Handle 10x data/models without rewrite",
    "8. Observability: Understand why models fail"
]

for principle in principles:
    print(f"   {principle}")

## 🚀 PART 8: Interactive Model Inference UI (Gradio)

In [ ]:
# %% [code]
import sys
import pandas as pd
from PIL import Image

class ModelInferenceUI:
    """Interactive UI for comparing our 9 models"""
    
    def __init__(self, all_models, prepared_data):
        self.models = all_models
        self.prepared_data = prepared_data
        self.results = all_results
        
        # Get best model for each dataset
        self.best_models = self._get_best_models()
        
        print("\n📱 PREPARING INTERACTIVE INFERENCE UI")
        print(f"   Total models: {len(self.models)}")
        print(f"   Best models per dataset: {list(self.best_models.keys())}")
    
    def _get_best_models(self):
        """Find best model for each dataset"""
        best_models = {}
        
        for dataset in ['mnist', 'fashion', 'cifar10']:
            # Find models for this dataset
            dataset_models = {k: v for k, v in self.results.items() if k[1] == dataset}
            
            if dataset_models:
                # Get model with highest accuracy
                best_key = max(dataset_models, key=lambda k: dataset_models[k]['test_accuracy'])
                best_model_name = f"{best_key[0]}_{best_key[1]}"
                best_models[dataset] = {
                    'name': best_model_name,
                    'model': self.models[best_model_name],
                    'accuracy': dataset_models[best_key]['test_accuracy']
                }
        
        return best_models
    
    def create_ui(self):
        """Create Gradio UI"""
        try:
            import gradio as gr
        except ImportError:
            print("Installing Gradio...")
            import subprocess
            subprocess.check_call([sys.executable, "-m", "pip", "install", "gradio"])
            import gradio as gr
        
        # Prepare sample images
        sample_images = self._prepare_sample_images()
        
        def predict(image, selected_dataset):
            """Predict using best model for selected dataset"""
            if image is None:
                return "Please upload or select an image", {}, None
            
            try:
                # Get best model for dataset
                if selected_dataset not in self.best_models:
                    return f"No model available for {selected_dataset}", {}, None
                
                model_info = self.best_models[selected_dataset]
                model = model_info['model']
                
                # Preprocess image
                processed = self._preprocess_image(image, selected_dataset)
                
                # Predict
                pred_labels, pred_probs = model.predict(processed)
                
                # Get class names
                classes = self.prepared_data[selected_dataset]['classes']
                pred_class = classes[pred_labels[0]]
                confidence = pred_probs[0][pred_labels[0]]
                
                # Create probabilities dict
                probabilities = {classes[i]: float(pred_probs[0][i]) for i in range(len(classes))}
                sorted_probs = dict(sorted(probabilities.items(), key=lambda x: x[1], reverse=True))
                
                # Create visualization
                fig = self._create_probability_plot(probabilities, pred_class, selected_dataset)
                
                result_text = (
                    f"**Prediction:** {pred_class} ({confidence:.1%})\n"
                    f"**Model:** {model_info['name']}\n"
                    f"**Dataset:** {selected_dataset.upper()}\n"
                    f"**Accuracy:** {model_info['accuracy']:.2%}"
                )
                
                return result_text, sorted_probs, fig
                
            except Exception as e:
                return f"Error: {str(e)}", {}, None
        
        def compare_all_models(image):
            """Compare all best models on same image"""
            if image is None:
                return pd.DataFrame(columns=["Dataset", "Model", "Prediction", "Confidence", "Accuracy"])
            
            results = []
            
            for dataset, model_info in self.best_models.items():
                try:
                    # Preprocess for this dataset
                    processed = self._preprocess_image(image, dataset)
                    
                    # Predict
                    pred_labels, pred_probs = model_info['model'].predict(processed)
                    
                    # Get class name
                    classes = self.prepared_data[dataset]['classes']
                    pred_class = classes[pred_labels[0]]
                    confidence = pred_probs[0][pred_labels[0]]
                    
                    results.append([
                        dataset.upper(),
                        model_info['name'],
                        pred_class,
                        f"{confidence:.1%}",
                        f"{model_info['accuracy']:.2%}"
                    ])
                except Exception as e:
                    results.append([dataset.upper(), model_info['name'], f"Error: {str(e)}", "N/A", "N/A"])
            
            return pd.DataFrame(results, columns=["Dataset", "Model", "Prediction", "Confidence", "Model Accuracy"])
        
        # Create UI
        with gr.Blocks(title="DNN Model Inference Dashboard", theme="soft") as demo:
            gr.Markdown("# 🧠 DNN Model Inference Dashboard")
            gr.Markdown("Compare best models from our 3×3 experiment matrix")
            
            with gr.Row():
                with gr.Column(scale=1):
                    dataset_dropdown = gr.Dropdown(
                        choices=["mnist", "fashion", "cifar10"],
                        value="mnist",
                        label="Select Dataset"
                    )
                    
                    image_input = gr.Image(
                        label="Upload or Draw Image",
                        type="numpy",
                        height=300,
                        sources=["upload", "webcam", "canvas"]
                    )
                    
                    gr.Markdown("### Sample Images")
                    
                    # Sample images for each dataset
                    for dataset_name, samples in sample_images.items():
                        gr.Markdown(f"**{dataset_name.upper()}**")
                        with gr.Row():
                            for label, img_array in samples:
                                def make_click_handler(img=img_array):
                                    return lambda: img
                                
                                gr.Button(label, size="sm").click(
                                    make_click_handler(),
                                    outputs=image_input
                                )
                    
                    predict_btn = gr.Button("Predict", variant="primary", size="lg")
                    compare_btn = gr.Button("Compare All Models", variant="secondary")
                
                with gr.Column(scale=2):
                    result_text = gr.Markdown("## Prediction will appear here")
                    
                    with gr.Row():
                        with gr.Column(scale=1):
                            prob_chart = gr.Plot(label="Probability Distribution")
                        with gr.Column(scale=1):
                            prob_json = gr.JSON(label="Probabilities")
                    
                    gr.Markdown("### Model Comparison")
                    comparison_table = gr.Dataframe(
                        label="Comparison of Best Models",
                        interactive=False,
                        wrap=True
                    )
                    
                    gr.Markdown("### Model Information")
                    model_info = gr.JSON(
                        value=self._get_model_info(),
                        label="Available Models"
                    )
            
            # Connect events
            predict_btn.click(
                fn=predict,
                inputs=[image_input, dataset_dropdown],
                outputs=[result_text, prob_json, prob_chart]
            )
            
            compare_btn.click(
                fn=compare_all_models,
                inputs=image_input,
                outputs=comparison_table
            )
        
        return demo
    
    def _prepare_sample_images(self):
        """Prepare sample images for each dataset"""
        sample_images = {}
        
        for dataset_name in ['mnist', 'fashion', 'cifar10']:
            # Get test data
            X_test, y_test = self.prepared_data[dataset_name]['test']
            classes = self.prepared_data[dataset_name]['classes']
            
            samples = []
            # Get one sample per class
            for class_idx in range(min(3, len(classes))):
                class_indices = np.where(y_test == class_idx)[0]
                if len(class_indices) > 0:
                    idx = class_indices[0]
                    img = X_test[idx]
                    
                    # Convert to uint8 for display
                    if img.dtype == np.float32 or img.dtype == np.float64:
                        img_display = (img * 255).astype(np.uint8)
                    else:
                        img_display = img.astype(np.uint8)
                    
                    label = f"{dataset_name[:3]}_{classes[class_idx][:10]}"
                    samples.append((label, img_display))
            
            sample_images[dataset_name] = samples
        
        return sample_images
    
    def _preprocess_image(self, image, dataset_name):
        """Preprocess image for specific dataset"""
        # Convert to numpy
        if hasattr(image, 'shape'):
            img_array = image
        else:
            img_array = np.array(image)
        
        # Handle RGBA
        if img_array.ndim == 3 and img_array.shape[-1] == 4:
            img_array = img_array[:, :, :3]
        
        # Resize
        target_size = (28, 28) if dataset_name in ['mnist', 'fashion'] else (32, 32)
        if img_array.shape[:2] != target_size:
            img_pil = Image.fromarray(img_array.astype(np.uint8))
            img_pil = img_pil.resize(target_size)
            img_array = np.array(img_pil)
        
        # Convert to grayscale for MNIST/Fashion
        if dataset_name in ['mnist', 'fashion'] and img_array.ndim == 3 and img_array.shape[-1] == 3:
            img_array = np.mean(img_array, axis=-1, keepdims=False)
        
        # Add batch dimension
        if img_array.ndim == 2:
            img_array = img_array[np.newaxis, ...]
        elif img_array.ndim == 3:
            img_array = img_array[np.newaxis, ...]
        
        # Apply dataset-specific preprocessing
        preprocessor = self.prepared_data[dataset_name]['preprocessor']
        processed = preprocessor.transform(img_array)
        
        return processed
    
    def _create_probability_plot(self, probabilities, predicted_class, dataset_name):
        """Create probability bar chart"""
        fig, ax = plt.subplots(figsize=(10, 6))
        
        classes = list(probabilities.keys())
        probs = list(probabilities.values())
        
        # Colors: highlight predicted class
        colors = ['lightblue' if cls != predicted_class else 'green' for cls in classes]
        
        bars = ax.bar(range(len(classes)), probs, color=colors, edgecolor='black')
        
        ax.set_xlabel('Classes')
        ax.set_ylabel('Probability')
        ax.set_title(f'Prediction Probabilities - {dataset_name.upper()}')
        ax.set_xticks(range(len(classes)))
        ax.set_xticklabels(classes, rotation=45, ha='right')
        ax.set_ylim([0, 1.1])
        ax.grid(True, alpha=0.3, axis='y')
        
        # Add probability labels
        for bar, prob in zip(bars, probs):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                   f'{prob:.1%}', ha='center', va='bottom', fontsize=8)
        
        plt.tight_layout()
        return fig
    
    def _get_model_info(self):
        """Get information about best models"""
        info = {}
        for dataset, model_info in self.best_models.items():
            model = model_info['model']
            info[dataset] = {
                'model_name': model_info['name'],
                'parameters': model.num_params,
                'layers': len(model.layers),
                'accuracy': f"{model_info['accuracy']:.2%}",
                'architecture': model.layer_sizes
            }
        return info
    
    def launch(self, share=False):
        """Launch the UI"""
        demo = self.create_ui()
        
        print("\n🚀 Launching Gradio UI...")
        print("   Local URL: http://localhost:7860")
        if share:
            print("   Public URL will be generated (expires in 72 hours)")
        
        try:
            demo.launch(share=share, server_name="0.0.0.0")
        except Exception as e:
            print(f"❌ Failed to launch UI: {e}")

# Create and launch UI
ui = ModelInferenceUI(all_models, prepared_data)

print("\n✅ UI ready. Uncomment the line below to launch:")
print("# ui.launch(share=False)  # Set share=True for public URL")

## 📋 PART 9: Pipeline Summary & Next Steps

In [ ]:
# %% [code]
import json

print("\n✅ LECTURE 3 COMPLETE: SUMMARY")
print("=" * 80)

# Save all results
summary = {
    "experiment_matrix": "3 architectures × 3 datasets = 9 models",
    "total_models": len(all_models),
    "datasets": list(prepared_data.keys()),
    "architectures": list(ARCHITECTURES.keys()),
    "best_models": {},
    "dataset_stats": dataset_stats,
    "total_training_time": sum(sum(m.history['epoch_times']) for m in all_models.values()),
    "total_parameters": sum(m.num_params for m in all_models.values()),
    "pipeline_created": True,
    "visualizations_created": len(list(VIZ_DIR.glob("*.png"))),
    "models_saved": len(list(MODELS_DIR.glob("*.pkl"))),
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
}

# Add best models
for dataset, model_info in ui.best_models.items():
    summary["best_models"][dataset] = {
        "model_name": model_info["name"],
        "accuracy": float(model_info["accuracy"]),
        "parameters": int(model_info["model"].num_params)
    }

# Save summary
summary_path = RESULTS_DIR / "lecture3_summary.json"
with open(summary_path, "w") as f:
    json.dump(summary, f, indent=2)

print(f"\n📊 EXPERIMENT SUMMARY:")
print(f"   • 9 models trained (3 architectures × 3 datasets)")
print(f"   • Total parameters: {summary['total_parameters']:,}")
print(f"   • Total training time: {summary['total_training_time']:.0f}s")
print(f"   • Models saved to: {MODELS_DIR}")
print(f"   • Visualizations saved to: {VIZ_DIR}")
print(f"   • Summary saved to: {summary_path}")

print("\n🏆 BEST MODELS PER DATASET:")
for dataset, model_info in ui.best_models.items():
    print(f"   • {dataset.upper():<12} → {model_info['name']:<25} ({model_info['accuracy']:.2%})")

print("\n🔜 NEXT STEPS:")
print("   1. Uncomment UI launch to interact with models")
print("   2. Analyze where DNNs fail (spatial patterns, translations)")
print("   3. Prepare for Lecture 4: Building CNNs from scratch")
print("   4. Convert this notebook to production pipeline (see structure below)")

print("\n🏗️ PRODUCTION PIPELINE STRUCTURE:")
print("config/           # YAML/JSON config files")
print("src/              # Source code")
print("  data/           # Data loading & preprocessing")
print("  models/         # Model definitions")
print("  training/       # Training loops")
print("  evaluation/     # Metrics & visualization")
print("  inference/      # Prediction & serving")
print("scripts/          # CLI entry points")
print("tests/            # Unit tests")
print("experiments/      # Experiment tracking")
print("models/           # Saved models")
print("results/          # Results & logs")

print("\n🎯 KEY ACHIEVEMENTS:")
print("1. Loaded 3 datasets directly from URLs (no libraries)")
print("2. Trained 9 models in 3×3 experiment matrix")
print("3. Visualized training dynamics for all models")
print("4. Demonstrated DNN limitations with complex data")
print("5. Designed production-ready pipeline architecture")
print("6. Built interactive UI for model comparison")
print("7. Prepared clear path to CNNs (next lecture)")

print("\n" + "=" * 80)
print("🎉 LECTURE 3 COMPLETE! Ready for CNNs in Lecture 4! 🎉")
print("=" * 80)